In [2]:
import cv2
import pytesseract
from PIL import Image
import numpy as np

In [3]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [6]:

def capture_image():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        cv2.imshow('Press Space to Capture', frame)
        if cv2.waitKey(1) & 0xFF == ord(' '):
            cv2.imwrite('captured_image.jpg', frame)
            break
    cap.release()
    cv2.destroyAllWindows()

def preprocess_image(image_path):
    # Load the image
    img = cv2.imread(image_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply thresholding
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
    
    # Remove noise
    kernel = np.ones((1, 1), np.uint8)
    processed_img = cv2.dilate(thresh, kernel, iterations=1)
    processed_img = cv2.erode(processed_img, kernel, iterations=1)
    
    # Save the processed image
    processed_image_path = 'processed_image.jpg'
    cv2.imwrite(processed_image_path, processed_img)
    
    return processed_image_path

def extract_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

def save_text_to_file(text, file_path):
    with open(file_path, 'w') as file:
        file.write(text)

capture_image()
image_path = 'captured_image.jpg'
processed_image_path = preprocess_image(image_path)
extracted_text = extract_text_from_image(processed_image_path)
print(extracted_text)

# Save the extracted text to a file
text_file_path = 'extracted_text.txt'
save_text_to_file(extracted_text, text_file_path)

ThelscienCerog
CelnRealization

fa
‘« a



In [13]:
import os
from pprint import pprint
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain.schema import Document
from langgraph.graph import END, StateGraph
from typing_extensions import TypedDict
from typing import List
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.document_loaders import FireCrawlLoader
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.document_loaders import FireCrawlLoader
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.docstore.document import Document

In [17]:
local_llm='deepseek-r1:14b'

In [30]:
# Import necessary libraries
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser

# Initialize the ChatOLLama model
llm = ChatOllama(model=local_llm, format="json", temperature=0)

# Define the prompt template correctly
prompt = PromptTemplate(
    template="""
    <|begin_of_text|><|start_header_id|>system<|end_header_id|> 
   You are a knowledgeable and compassionate doctor providing clear guidance on a patient's prescribed medicine.
    Based on the given medicine description, provide:
    
    1. **When to Take It:** Clearly explain the best time and conditions under which the medicine should be taken (e.g., with food, before sleep, on an empty stomach).
    2. **Precautions:** List important things the patient should be aware of before taking the medicine (e.g., allergies, interactions, conditions that may worsen).
    3. **Side Effects:** Describe common and severe side effects in simple terms, and when the patient should seek medical attention.
    4. **Additional Advice:** Provide any extra tips to ensure safe and effective use of the medicine.

    Be clear, friendly, and reassuring so that the patient fully understands their medication.

    Here is the medicine description:
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    {context}
    Here is the user question: {question}
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "context"]
)

# Initialize the retrieval grader with a correct pipeline
retrieval_grader = prompt | llm | JsonOutputParser()

# Define the user question and sample context
question = "Is there a possibility to get diabetics while being pregnant?"
context = "Gestational diabetes is a condition where high blood sugar develops during pregnancy."

# Use the retrieval grader to assess the context's relevance
result = retrieval_grader.invoke({"question": question, "context": context})

# Print the result
print(result)


{'when_to_take_it': {'description': 'Take this medicine exactly as prescribed by your doctor. Do not change the dosage or stop taking it without consulting your healthcare provider.'}, 'precautions': {'description': 'Inform your doctor if you have any allergies, especially to other diabetes medications. Also, let them know about any other medications or supplements you are taking. If you experience severe side effects like difficulty breathing, swelling of the face, lips, or tongue, seek immediate medical attention.'}, 'side_effects': {'description': 'Common side effects may include hypoglycemia (low blood sugar), which can cause symptoms like shakiness, sweating, dizziness, and confusion. Rare but serious side effects might involve an allergic reaction, leading to difficulty breathing or swelling of the face.'}, 'additional_advice': {'description': 'Monitor your blood sugar levels regularly as directed by your doctor. Maintain a healthy diet and exercise routine. Always carry a fast-a